In [1]:
import numpy as np
import pandas as pd
import random
import time
import scipy as sp
from scipy.sparse.linalg import cg
from sklearn.metrics.pairwise import rbf_kernel

In [2]:
from data_datasets import higgs, susy, cod_rna
N =5000
data = "susy"
# data = "higgs"
# data = "cod_rna"
####################

if data == "higgs":
    X_train, X_test, y_train, y_test = higgs(N)
elif data == "susy":
    X_train, X_test, y_train, y_test = susy(N)
elif data == "cod_rna":
    X_train, X_test, y_train, y_test = cod_rna(N)

print("\nDataset:", data)
print("--------\nShape train data:", X_train.shape)
print("Shape test data:", X_test.shape)


Dataset: susy
--------
Shape train data: (5000, 18)
Shape test data: (5000, 18)


In [ ]:
eps = np.finfo(float).eps
eps

In [ ]:
def nystrom(K, rank):
    N = K.shape[0]
    Omega = np.random.randn(N, rank)
    Y = K@Omega  # matvec
    eps = np.finfo(float).eps
    shift = np.linalg.norm(Y, "fro")*eps
    Y_shift = Y+shift*Omega
    C = np.linalg.cholesky(Omega.T@Y_shift)
    B = Y_shift @ np.linalg.inv(C)

    U, D, VT = np.linalg.svd(B, full_matrices=False)
    Sigma = np.diag(D)
    Lamda = np.maximum(0, Sigma@Sigma-shift*np.eye(rank))

    return U, Lamda

In [ ]:
from low_rank_methods import nystrom
mu = .1
rank = 20

N, d = X_train.shape
I = np.eye(N)
i = np.eye(rank)

K = rbf_kernel(X_train, gamma=.1)
K_reg=K+mu*I
U = nystrom(K, rank)[0]
D = nystrom(K, rank)[1]
print(U.shape, D.shape)

In [ ]:
from utils import diag_inv

residuals_pre = []
b = y_train
B = diag_inv(D+mu)
lambda_l = D[-1, -1]

P_nys = U@(D+mu)@U.T/(lambda_l+mu)+I-U@U.T
P_nys_inv = (lambda_l+mu)*U@B@U.T+I-U@U.T


def callback(x): return residuals_pre.append(
    np.linalg.norm(K_reg @ x - b) )
# / np.linalg.norm(b)

alpha_pre, info = cg(K_reg, b, M=P_nys_inv, callback=callback, tol=1e-3)
len(residuals_pre)

In [ ]:
residuals = []
b = y_train


def callback(x): return residuals.append(
    np.linalg.norm(K_reg @ x - b) )
# / np.linalg.norm(b)


alpha, info = cg(K_reg, b, callback=callback, tol=1e-3)
len(residuals)

In [ ]:
import copy
import scipy.linalg as la



def CG(A, b, P_inv=None, matvec=None, tol=1e-5, x0=None, max_iter=None):
    """
    Conjugate Gradient Method for solving Ax = b
    :param A: matrix
    :param b: vector
    :param tol: tolerance (default: 1e-5)
    :param x0: initial guess (default: Zero vector)
    :param max_iter: Maximum number of iterations (default: Dimention of A)
    :param P_inv:  The inveresidualse of Preconditioner (default: None for CG)
    :return: x, residuals
    """

    N = len(b)
    residuals = []
    dot = np.dot

    if max_iter is None:
        max_iter = N

    def matvec(M, u):
        return M@u

    x = np.zeros(N) if x0 == None else x0
    r = b - matvec(A, x) if x.any() else b.copy()
    p = r
    z = np.zeros(N)

    if (P_inv is None):
        method = "cg"
    else:
        method = "pcg"
        z = P_inv@r

    # Main Loop
    i = 0
    while i < max_iter or la.norm(r) > tol:
        residuals.append(np.linalg.norm(r))

        if la.norm(r) < tol:
            return x, np.array(residuals)

        v = matvec(A, p)
        v = np.nan_to_num(v)

        if method == "cg":
            alpha = dot(r, r) / dot(p, v)
            alpha = np.nan_to_num(alpha)


        elif method == "pcg":
            alpha = dot(r, z) / dot(p, v)
            alpha = np.nan_to_num(alpha)


        # Update x and r
        x = x + alpha * p
        x = np.nan_to_num(x)

        r_new = r - alpha * v
        r_new = np.nan_to_num(r_new)


        # Update x and r

        if method == "cg":
            beta = dot(r_new, r_new)/dot(r, r)
            beta = np.nan_to_num(beta)

        elif method == "pcg":
            z_new = P_inv@r_new
            beta = dot(z_new, r_new)/dot(z, r)
            beta = np.nan_to_num(beta)


        p = r_new + beta * p
        p = np.nan_to_num(p)

        r = r_new
        i = i+1

    return x, np.array(residuals)

In [ ]:
# """Show plot with convergence profile - normalised residual vector vs iteration."""
import matplotlib.pyplot as plt
plt.title('Pre KRR method convergence profile.')
plt.ylabel('Convergence (residual norm)')
plt.xlabel('Iterations')

m_res = residuals
plt.semilogy(range(len(m_res)), m_res, 'b--')

plt.legend(['Total iter = ' + str(len(m_res))])
# plt.show()